### Test

Testeo cómo importar otros notebooks

Conclusiones:  
* Puedo importar desde dos notebooks distintos las mismas variables. La que importa va a ser la última que se carga porque pisa la anterior
* Una vez que importo, por más que corra la celda devuelta no se van a volver a importar. Se ve que ya compiló las funciones dentro del notebook que importo y no lo vuelve a hacer. Esto implica que si hago algún cambio en el otro archivo no lo voy a ver por más que lo importe devuelta. Para sobreescribir los cambios hay que reiniciar el kernel
* Puedo importar cíclicamente: test1 importa a test2 que importa a test1 y así sucesivamente, pero el proceso se corta cuando en ambos archivos ya están compiladas las funciones del otro

In [1]:
#Código necesario para importar funciones de otros notebooks .ipynb

import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path

class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod


class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

sys.meta_path.append(NotebookFinder())

#Ejemplo
# from derivada_reacciones import *

In [2]:
#Importo notebooks

from test2 import *
print(alpha)
from test3 import *

importing Jupyter notebook from test2.ipynb
importing Jupyter notebook from test3.ipynb
importing Jupyter notebook from test1.ipynb
2
2
1
2
2
1


In [ ]:
print("Hola")

In [3]:
print(alpha)
print(beta)
print(alpha)

2
1
2


In [5]:
asigno()

NameError: name 'asigno' is not defined